In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/project-m3-notanagi/notebooks/sft_training

/content/drive/MyDrive/project-m3-notanagi/notebooks/sft_training


In [3]:
!pip install sentencepiece
!pip install -U bitsandbytes
!pip install -U git+https://github.com/huggingface/transformers.git 
!pip install -U git+https://github.com/huggingface/peft.git
!pip install -U git+https://github.com/huggingface/accelerate.git
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 10.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-ek8wzumk
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-ek8wzumk
  Resolved https://github.com/huggingface/transformers.git to commit 0d217f428fa5b84fcde9c31ef1c95e4215856f7e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 5.7 MB/s eta 0:00:

In [4]:
import sys
MODULE_PATHS = ["../../modules"]
for module_path in MODULE_PATHS:
    if module_path not in sys.path:
        sys.path.append(module_path)

import json


import torch
from fine_tuning import build_finetuning_hf_dataset

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from datasets import load_dataset
from transformers import BitsAndBytesConfig


%load_ext autoreload
%autoreload 2


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('http'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

In [5]:
model_id = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, device_map="auto")

In [6]:
from peft import TaskType

#model.gradient_checkpointing_enable()
#model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)
model = get_peft_model(model, config)

# print number of trainable parameters
model.print_trainable_parameters()

trainable params: 1,769,472 || all params: 249,347,328 || trainable%: 0.7096414524241463


In [7]:
# Verifying the datatypes.
dtypes = {}
for _, p in model.named_parameters():
    dtype = p.dtype
    if dtype not in dtypes:
        dtypes[dtype] = 0
    dtypes[dtype] += p.numel()
total = 0
for k, v in dtypes.items():
    total += v
for k, v in dtypes.items():
    print(k, v, v / total)

torch.float32 249347328 1.0


In [8]:
max_input_length = 512
# change max input length of tokenizer to max_input_length
tokenizer.model_max_length = max_input_length
with open("../../submission_items/gen_dataset_NotAnAGI.json", "rb") as reader:
    dataset = build_finetuning_hf_dataset(json.load(reader), tokenizer=tokenizer, max_input_length=max_input_length)

Map:   0%|          | 0/10647 [00:00<?, ? examples/s]

Map:   0%|          | 0/10647 [00:00<?, ? examples/s]

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10114
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 533
    })
})

In [10]:
dataset["train"][1]["labels"]

tensor([  304,  1731,     6,   752,    31,     7,  1431,    91,     8,  3893,
           21,  1514,   371,  3229,    10,  1514,  3229,   377,  3274,     3,
            2,  3357,   107,   115,   115,     2,   427,     2,   834,   476,
            3,     2,  2152,  1027,   599,   476,    61,  3274,     3,     2,
         3357,   107,   115,   115,     2,   427,     2,   834,   476,     3,
            2,  2152,     3,     2,    77,    17,     3,    26,     4,     3,
           15,     2,    18,     2,  9880,     2,   599,   476,    18,     4,
           61,     2,   357,     2,   357,     2,  2962,    40,    17,     9,
            2,   276,   599,     4,    61,  1514,  3229,   852,     6,   752,
           31,     7,   240,     8, 21875,    13,  1514,   371,  3229,    28,
         1445,    12,  1514,     2,  2962,    40,    17,     9,     2,  2292,
            2,  3229,    10,  1514,  3229,     3,     2,   346,   122,    77,
            2,   138,  6962,    26,     2,     3,     2,  1893, 

In [11]:
len(dataset["train"][1]["labels"])

512

# Before training

In [ ]:
def generate_example(model, tokenizer, dataset, num_examples=5):
    for i in range(num_examples):
        example_input = dataset["test"][i]["input_ids"].to("cuda")
        example_label = dataset["test"][i]["labels"].to("cuda")

        output = model.generate(
            input_ids=example_input.unsqueeze(0),
            max_new_tokens=1000,
            temperature=0.9,
            top_p=0.9,
            num_return_sequences=1,
            #repetition_penalty=100.0

        )

        print("RESULT: ", i)
        print(tokenizer.decode(output[0], skip_special_tokens=True))
        # decode input and label
        print("INPUT: ", i)
        print(tokenizer.decode(example_input, skip_special_tokens=True))
        print("LABEL: ", i)
        print(tokenizer.decode(example_label, skip_special_tokens=True))
        print("--------------------------------------------------------")

generate_example(model, tokenizer, dataset, num_examples=5)

RESULT:  0
The redshift of the galaxies is about 5 and the wavelength range is 6000 to 7000 Angström. The wavelength range of the ultraviolet filter is about 6000 to 7000 Angström. The wavelength range of the infrared filter is about 7000 to 7000 Angström. The wavelength range of the ultraviolet filter is about 7000 to 7000 Angström. The wavelength range of the ultraviolet filter is about 6000 to 7000 Angström. The wavelength range of the ultraviolet filter is about 7000 to 7000 Angström. The wavelength range of the ultraviolet filter is about 7000 to 7000 Angström. The wavelength range of the ultraviolet filter is about 7000 to 7000 Angström. The wavelength range of the ultraviolet filter is about 7000 to 7000 Angström. The wavelength range of the ultraviolet filter is about 7000 to 7000 Angström. The wavelength range of the ultraviolet filter is about 7000 to 7000 Angström. The wavelength range of the ultraviolet filter is about 7000 to 7000 Angström. The wavelength range of the ultr

In [ ]:
from transformers import DataCollatorForSeq2Seq
training_args = TrainingArguments(
    #auto_find_batch_size=True,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=4,
    learning_rate=1e-3,
    #learning_rate=5e-5,
    fp16=False,
    save_total_limit=4,
    logging_steps=5,
    eval_steps=100,
    output_dir="./outputs",
    save_strategy='epoch',
    #optim="paged_adamw_8bit",
    lr_scheduler_type = 'cosine',
    warmup_ratio = 0.1,
)
class ConversationTrainer(Trainer):    
    def compute_loss(self, model, inputs, return_outputs=False):
        assert return_outputs == False, "we did not implement return_outputs=True in RewardTrainer.compute_loss"
        
        #print("inputs: ", inputs)
        input_ids = inputs["input_ids"]
        labels = inputs["labels"]
        #print("labels: ", labels)
        print("len inputs: ", input_ids.shape)
        print("len labels: ", labels.shape)
        loss, outputs = Trainer.compute_loss(self, model, inputs, return_outputs=True)
        print("loss: ", loss)
        print("outputs: ", outputs)
        wefwefw
        return loss



        


# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    padding="max_length",
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)


trainer = Trainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    args=training_args,
    data_collator=data_collator
)



In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
5,2.146700
10,2.309300
15,2.138300
20,2.294100
25,2.153200
30,2.331300
35,2.098300
40,2.157900
45,2.143300
50,2.151900


TrainOutput(global_step=2528, training_loss=1.7502318441490583, metrics={'train_runtime': 9173.8973, 'train_samples_per_second': 4.41, 'train_steps_per_second': 0.276, 'total_flos': 2.7912792198610944e+16, 'train_loss': 1.7502318441490583, 'epoch': 4.0})

# Checkpoint 1

In [31]:
from peft import PeftModel, PeftConfig
peft_model_id = "../../../outputs/checkpoint-632"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path, device_map="auto")
model1 = PeftModel.from_pretrained(model, peft_model_id)
model1.eval()

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 768)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 768)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): Linear(
                    in_features=768, out_features=768, bias=False
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out_features=16, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=16, out_features=768, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
                    (lora_embedding_B):

In [37]:
def generate_example(gen_model, tokenizer, dataset, num_examples=5):
    for i in range(num_examples):
        example_input = dataset["test"][i]["input_ids"].to("cuda")
        example_label = dataset["test"][i]["labels"].to("cuda")

        output = gen_model.generate(
            input_ids=example_input.unsqueeze(0),
            max_new_tokens=1000,
            temperature=0.9,
            top_p=0.9,
            num_return_sequences=1,
            repetition_penalty=100.0

        )

        print("RESULT: ", i)
        print(tokenizer.decode(output[0], skip_special_tokens=True))
        # decode input and label
        print("INPUT: ", i)
        print(tokenizer.decode(example_input, skip_special_tokens=True))
        print("LABEL: ", i)
        print(tokenizer.decode(example_label, skip_special_tokens=True))
        print("--------------------------------------------------------")

generate_example(model1, tokenizer, dataset, num_examples=5)

RESULT:  0
The correct answer is (A) Because of the cosmological shift towards smaller wavelengths at high redshift, one has to observe the H alpha emission line in an ultraviolet filter. This means that there are no visible signs of stellar formation in very old galaxie with the redshift of about 5. Therefore, we need to use the R filter covering the wavelength range from 6000–7000 Angström.
INPUT:  0
Question: We look for signs of the stellar formation in very old galaxies with the redshift of about 5. So one wants to observe the H alpha emission line in the gas clouds made of ionised hydrogen. The R filter (covering the wavelength range from 6000 to 7000 Angström) seems to be the right filter to use. Unfortunately, there is nothing in the images. Why? (A) Because of the cosmological shift towards smaller wavelengths at high redshift, one has to observe the H alpha line in an ultraviolet filter. (B) Because of the shift towards longer wavelengths at high redshift, one has to observe 

# checkpoint 4

In [35]:
from peft import PeftModel, PeftConfig
peft_model_id = "../../../outputs/checkpoint-2528"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path, device_map="auto")
model4 = PeftModel.from_pretrained(model, peft_model_id)
model4.eval()

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 768)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 768)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): Linear(
                    in_features=768, out_features=768, bias=False
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out_features=16, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=16, out_features=768, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
                    (lora_embedding_B):

In [38]:
generate_example(model4, tokenizer, dataset, num_examples=5)

RESULT:  0
The correct answer is (A) Because of the cosmological shift towards smaller wavelengths at high redshift, one has to observe the H alpha emission line in an ultraviolet filter. This means that there are no visible signs of stellar formation in very old galaxie with the redshift of about 5. Therefore, we need to use the R filter covering the wavelength range from 6000–7000 Angström.
INPUT:  0
Question: We look for signs of the stellar formation in very old galaxies with the redshift of about 5. So one wants to observe the H alpha emission line in the gas clouds made of ionised hydrogen. The R filter (covering the wavelength range from 6000 to 7000 Angström) seems to be the right filter to use. Unfortunately, there is nothing in the images. Why? (A) Because of the cosmological shift towards smaller wavelengths at high redshift, one has to observe the H alpha line in an ultraviolet filter. (B) Because of the shift towards longer wavelengths at high redshift, one has to observe 